In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# bbox.txt 파일들만 모아 불러오기
file_dir = '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt'

bbox_list = []
for (dirpath, dirnames, filenames) in os.walk(file_dir):
    for file in filenames:
        combined_path = os.path.join(dirpath, file)
        if '.DS_Store' in combined_path:
            continue
        elif 'bbox.txt' in combined_path:
            bbox_list += [combined_path]
        else:
            continue

In [ ]:
len(bbox_list)

50 50


In [ ]:
bbox_list

['/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/morning/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/rain/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/overcast/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/15-deg-right/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/30-deg-left/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/sunset/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/clone/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/fog/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/15-deg-left/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene06/30-deg-right/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textgt/Scene01/morning/bbox.txt',
 '/Users/ksh/Documents/Hackathon/데이터셋/vkitti_2.0.3_textg

In [ ]:
# bbox.txt 파일들 모두 합치기
bbox1 = pd.read_csv(bbox_list[0], delimiter=' ')

for i in range(1, len(bbox_list)):
    bbox2 = pd.read_csv(bbox_list[i], delimiter=' ')
    bbox1 = pd.concat([bbox1, bbox2])
    bbox1 = bbox1[['trackID', 'left', 'right', 'top', 'bottom', 'truncation_ratio', 'occupancy_ratio']]

bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio
0,0,300,533,189,292,0.0,0.502104
1,0,265,502,189,292,0.0,0.504649
2,0,233,487,189,300,0.0,0.513833
3,0,196,454,189,300,0.0,0.513688
4,0,152,434,190,312,0.0,0.579090
...,...,...,...,...,...,...,...
2870,16,118,160,198,223,0.0,0.591429
2871,17,47,100,200,226,0.0,0.602322
2872,18,147,180,197,219,0.0,0.552342
2873,19,83,125,200,221,0.0,0.515873


In [ ]:
# trackID -> className으로 바꿔주기 위한 csv 파일 불러오기
label_index = pd.read_csv('/Users/ksh/Documents/Hackathon/데이터셋/CAR_VAN_TRUCK.csv')
label_index

,index,className
0,0,Car
1,1,Car
2,10,Car
3,100,Car
4,101,Car
...,...,...
127,94,Car
128,95,Car
129,96,Car
130,97,Car


In [ ]:
label_index.rename(columns={'index' : 'trackID'}, inplace=True)
label_index

,trackID,className
0,0,Car
1,1,Car
2,10,Car
3,100,Car
4,101,Car
...,...,...
127,94,Car
128,95,Car
129,96,Car
130,97,Car


In [ ]:
# trackID 기준으로 클래스 라벨 데이터프레임과 병합
bbox1 = pd.merge(bbox1, label_index, on='trackID')
bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className
0,0,300,533,189,292,0.0,0.502104,Car
1,0,265,502,189,292,0.0,0.504649,Car
2,0,233,487,189,300,0.0,0.513833,Car
3,0,196,454,189,300,0.0,0.513688,Car
4,0,152,434,190,312,0.0,0.579090,Car
...,...,...,...,...,...,...,...,...
532889,131,289,327,164,182,0.0,0.738304,Car
532890,131,302,340,164,182,0.0,0.735380,Car
532891,131,295,334,164,182,0.0,0.723647,Car
532892,131,307,346,164,182,0.0,0.716524,Car


In [ ]:
bbox1['className'].unique()

array(['Car', 'Van', 'Truck'], dtype=object)

In [ ]:
# class별 객체 수 구하기
bbox1['className'].value_counts()

Car      479058
Van       48321
Truck      5515
Name: className, dtype: int64

In [ ]:
# occ
bbox1['occluded_ratio'] = 1 - bbox1['occupancy_ratio']
bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className,occluded_ratio
0,0,300,533,189,292,0.0,0.502104,Car,0.497896
1,0,265,502,189,292,0.0,0.504649,Car,0.495351
2,0,233,487,189,300,0.0,0.513833,Car,0.486167
3,0,196,454,189,300,0.0,0.513688,Car,0.486312
4,0,152,434,190,312,0.0,0.579090,Car,0.420910
...,...,...,...,...,...,...,...,...,...
532889,131,289,327,164,182,0.0,0.738304,Car,0.261696
532890,131,302,340,164,182,0.0,0.735380,Car,0.264620
532891,131,295,334,164,182,0.0,0.723647,Car,0.276353
532892,131,307,346,164,182,0.0,0.716524,Car,0.283476


In [ ]:
occ_list = [
    (bbox1['occluded_ratio'] <= 0.05),
    (bbox1['occluded_ratio'] > 0.05) & (bbox1['occluded_ratio'] <= 0.33),
    (bbox1['occluded_ratio'] > 0.33) & (bbox1['occluded_ratio'] <= 0.66),
    (bbox1['occluded_ratio'] > 0.66)]

level = [0, 1, 2, 3]

bbox1['occluded_level'] = np.select(occ_list, level, default='Not Specified')
bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className,occluded_ratio,occluded_level
0,0,300,533,189,292,0.0,0.502104,Car,0.497896,2
1,0,265,502,189,292,0.0,0.504649,Car,0.495351,2
2,0,233,487,189,300,0.0,0.513833,Car,0.486167,2
3,0,196,454,189,300,0.0,0.513688,Car,0.486312,2
4,0,152,434,190,312,0.0,0.579090,Car,0.420910,2
...,...,...,...,...,...,...,...,...,...,...
532889,131,289,327,164,182,0.0,0.738304,Car,0.261696,1
532890,131,302,340,164,182,0.0,0.735380,Car,0.264620,1
532891,131,295,334,164,182,0.0,0.723647,Car,0.276353,1
532892,131,307,346,164,182,0.0,0.716524,Car,0.283476,1


In [ ]:
bbox1['occluded_level'].value_counts()

3    276503
2    165592
1     88934
0      1865
Name: occluded_level, dtype: int64

In [ ]:
## BBOX 확인

image_size = 1242*375

bbox1['bbox'] = (bbox1['right']-bbox1['left'])*(bbox1['bottom']-bbox1['top']) / image_size
bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className,occluded_ratio,occluded_level,bbox
0,0,300,533,189,292,0.0,0.502104,Car,0.497896,2,0.051528
1,0,265,502,189,292,0.0,0.504649,Car,0.495351,2,0.052412
2,0,233,487,189,300,0.0,0.513833,Car,0.486167,2,0.060535
3,0,196,454,189,300,0.0,0.513688,Car,0.486312,2,0.061488
4,0,152,434,190,312,0.0,0.579090,Car,0.420910,2,0.073868
...,...,...,...,...,...,...,...,...,...,...,...
532889,131,289,327,164,182,0.0,0.738304,Car,0.261696,1,0.001469
532890,131,302,340,164,182,0.0,0.735380,Car,0.264620,1,0.001469
532891,131,295,334,164,182,0.0,0.723647,Car,0.276353,1,0.001507
532892,131,307,346,164,182,0.0,0.716524,Car,0.283476,1,0.001507


In [ ]:
bbox_list = [
    (bbox1['bbox'] <= 0.0001),
    (bbox1['bbox'] > 0.0001) & (bbox1['bbox'] <= 0.001),
    (bbox1['bbox'] > 0.001) & (bbox1['bbox'] <= 0.01),
    (bbox1['bbox'] > 0.01) & (bbox1['bbox'] <= 0.1),
    (bbox1['bbox'] > 0.1)]

bb_level = [0, 1, 2, 3, 4]

bbox1['bbox_level'] = np.select(bbox_list, bb_level, default='Not Specified')
bbox1

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className,occluded_ratio,occluded_level,bbox,bbox_level
0,0,300,533,189,292,0.0,0.502104,Car,0.497896,2,0.051528,3
1,0,265,502,189,292,0.0,0.504649,Car,0.495351,2,0.052412,3
2,0,233,487,189,300,0.0,0.513833,Car,0.486167,2,0.060535,3
3,0,196,454,189,300,0.0,0.513688,Car,0.486312,2,0.061488,3
4,0,152,434,190,312,0.0,0.579090,Car,0.420910,2,0.073868,3
...,...,...,...,...,...,...,...,...,...,...,...,...
532889,131,289,327,164,182,0.0,0.738304,Car,0.261696,1,0.001469,2
532890,131,302,340,164,182,0.0,0.735380,Car,0.264620,1,0.001469,2
532891,131,295,334,164,182,0.0,0.723647,Car,0.276353,1,0.001507,2
532892,131,307,346,164,182,0.0,0.716524,Car,0.283476,1,0.001507,2


In [ ]:
bbox1['bbox_level'].value_counts()

2    199568
1    137647
0     92837
3     90276
4     12566
Name: bbox_level, dtype: int64